In [44]:
import pysd

In [45]:
model = pysd.load('bass_diffusion.py')

In [46]:
model.run()

,TIME,New Reported Cases,Infected,Susceptible,Contact Frequency,Contacts Between Infected and Uninfected Persons,Cumulative Reported Cases,Infection Rate,Infectivity,Probability of Contact with Infected Person,Susceptible Contacts,Total Population,FINAL TIME,INITIAL TIME,SAVEPER,TIME STEP
0.000,0.000,0.350000,1.000000,10000.000000,7,7.000000,0.000000,0.350000,0.05,0.000100,70000.000000,10000,35,0,0.125,0.125
0.125,0.125,0.365311,1.043750,9999.956250,7,7.306218,0.043750,0.365311,0.05,0.000104,69999.693750,10000,35,0,0.125,0.125
0.250,0.250,0.381291,1.089414,9999.910586,7,7.625829,0.089414,0.381291,0.05,0.000109,69999.374103,10000,35,0,0.125,0.125
0.375,0.375,0.397971,1.137075,9999.862925,7,7.959418,0.137075,0.397971,0.05,0.000114,69999.040473,10000,35,0,0.125,0.125
0.500,0.500,0.415380,1.186822,9999.813178,7,8.307596,0.186822,0.415380,0.05,0.000119,69998.692248,10000,35,0,0.125,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34.500,34.500,201.600437,9387.410960,613.589040,7,4032.008734,9386.410960,201.600437,0.05,0.938741,4295.123278,10000,35,0,0.125,0.125
34.625,34.625,193.839682,9412.611015,588.388985,7,3876.793650,9411.611015,193.839682,0.05,0.941261,4118.722896,10000,35,0,0.125,0.125
34.750,34.750,186.335765,9436.840975,564.159025,7,3726.715302,9435.840975,186.335765,0.05,0.943684,3949.113174,10000,35,0,0.125,0.125
34.875,34.875,179.083598,9460.132946,540.867054,7,3581.671967,9459.132946,179.083598,0.05,0.946013,3786.069380,10000,35,0,0.125,0.125
